Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты.

КРИТЕРИИ ОЦЕНИВАНИЯ:

* 1 балл	Верно выделены три столбца-признака для обучения, выбранные RFE.
* 1 балл	Верно выделены три столбца-признака для обучения, выбранные SelectKBest.
* 3 балла	Обучена регрессия на первых трёх столбцах, оценено качество модели на тесте.
* 3 балла	Обучена регрессия на вторых трёх столбцах, оценено качество модели на тесте.
* 2 балла	Произведено сравнение выбранных метрик в форме комментария. Дан ответ на вопрос «Какой метод отбора признаков показал наилучший результат на тестовой выборке?» (в текстовой ячейке).

Максимальное количество баллов за выполнение задания — 10.

In [9]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [10]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [11]:
data = pd.read_excel('data_ford_price.xlsx') 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [12]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

# Выделим три столбца-признака для обучения, выбранные RFE.

In [13]:
estimator_rfe = LinearRegression()
selector_rfe = RFE(estimator_rfe, n_features_to_select=3, step=1)
selector_rfe = selector_rfe.fit(X_train, y_train)
 
selector_rfe.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

# Выделим три столбца-признака для обучения, выбранные SelectKBest.

In [14]:
selector_kbest = SelectKBest(f_regression, k=3)
selector_kbest.fit(X_train, y_train)
 
selector_kbest.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [16]:
print('RFE most important features: ', selector_rfe.get_feature_names_out())
print('SelectKBest most important features: ', selector_kbest.get_feature_names_out())

RFE most important features:  ['year' 'cylinders' 'lat']
SelectKBest most important features:  ['year' 'cylinders' 'odometer']


# Обучим регрессия на первых трёх столбцах(RFE), оцененим качество модели на тесте.

In [26]:
data.dropna(inplace=True)

X_rfe = data[selector_rfe.get_feature_names_out()]
y = data['price']

X_train_rfe, X_test_rfe, y_train, y_test = train_test_split(X_rfe, y, test_size=0.2, random_state=42)

model_rfe = LinearRegression()
model_rfe.fit(X_train_rfe, y_train)
y_predict = model_rfe.predict(X_test_rfe)

display(f'Test MAE (rfe): {mean_absolute_error(y_test, y_predict):.0f}')
display(f'Test R^2 (rfe): {r2_score(y_test, y_predict):.3f}')

'Test MAE (rfe): 5280'

'Test R^2 (rfe): 0.562'

# Обучим регрессию на вторых трёх столбцах, оценено качество модели на тесте.

In [25]:
data.dropna(inplace=True)

X_kbest = data[selector_kbest.get_feature_names_out()]
y = data['price']

X_train_kbest, X_test_kbest, y_train, y_test = train_test_split(X_kbest, y, test_size=0.2, random_state=42)

model_kbest = LinearRegression()
model_kbest.fit(X_train_kbest, y_train)
y_predict = model_kbest.predict(X_test_kbest)

display(f'Test MAE (kbest): {mean_absolute_error(y_test, y_predict):.0f}')
display(f'Test R^2 (kbest): {r2_score(y_test, y_predict):.3f}')

'Test MAE (kbest): 4925'

'Test R^2 (kbest): 0.598'

# Вывод

Вывод: 
MAE - показывает насколько в среднем модель ошибается,чем ближе к нулю, тем лучше, MAE ниже у второй модели.

R2 - показывает, насколько наша модель лучше, чем если бы все предсказания были средним по правильным ответам. R2 для воторой модели лучше.

Наилучший результат по тестовой выборке показа метод выбора признаков на основе фильтров